# 1 Setup

## 1.1 Import Modules

In [51]:
import arcpy
from os.path import join as join_path
from calendar import monthrange as month_range
from pandas import DataFrame
from tqdm import tqdm

## 1.2 Set Workplace and Environmental Variables

In [52]:
root = "D:/Repos/Air Pollution Exposure in Xi'an/Projects/Publish"

aprx = arcpy.mp.ArcGISProject('CURRENT')

arcpy.env.overwriteOutput = True

## 1.3 Functions

In [108]:
def apply_border_renderer(border, symbol, color = None, width = None):
    sym = border.symbology
    sym.renderer.symbol.applySymbolFromGallery(symbol)
    
    if color != None:
        sym.renderer.symbol.outlineColor = color
    if width != None:
        sym.renderer.symbol.outlineWidth = width
    
    border.symbology = sym

In [109]:
def apply_raster_colorizer(raster, color_ramp, stretch_min, stretch_max):
    sym = raster.symbology

    colorRamp = aprx.listColorRamps(color_ramp)[0]
    sym.colorizer.colorRamp = colorRamp
    sym.colorizer.stretchType = 'Custom'

    raster.symbology = sym

    raster_cim = raster.getDefinition('V3')

    colorizer = raster_cim.colorizer
    colorizer.statsType = 'GlobalStats'
    colorizer.stretchStats.min = stretch_min
    colorizer.stretchStats.max = stretch_max
    colorizer.stretchClasses[0].value = stretch_min
    colorizer.stretchClasses[2].value = stretch_max
    colorizer.stretchClasses[0].label = f'{stretch_min}'
    colorizer.stretchClasses[2].label = f'{stretch_max}'
    colorizer.useCustomStretchMinMax = True

    raster_cim.colorizer = colorizer
    raster.setDefinition(raster_cim)

In [110]:
def apply_feature_renderer(feature, color_ramp, classification_field, break_count):
    sym = feature.symbology

    sym.updateRenderer('GraduatedColorsRenderer')
    sym.renderer.classificationField = classification_field
    sym.renderer.breakCount = break_count

    colorRamp = aprx.listColorRamps(color_ramp)[0]
    sym.renderer.colorRamp = colorRamp

    feature.symbology = sym

## 2 Visualization

## 2.1 PM 2.5 Concentration: Calendar

In [13]:
daily_mean_values = []

with tqdm(total = 365 * 3) as pbar:
    for month in range(1, 13):
        for day in range(1, month_range(2021, month)[1] + 1):
            date_string = '%s%s%s' % (2021, '%02d' % month, '%02d' % day)
            mean_values = {
                'date': f'{month}/{day}/2021'
            }
            for type_ in ['wholeDay', 'daytime', 'nighttime']:
                raster_name = join_path(root, 'Kriging.gdb', f'DailyKriging_{date_string}_{type_}')
                if arcpy.Exists(raster_name):
                    raster = arcpy.Raster(raster_name)
                    mean_values[type_] = raster.mean
                pbar.update(1)
            daily_mean_values.append(mean_values)

100%|██████████| 1095/1095 [03:52<00:00,  4.71it/s]﻿


In [17]:
excel_filename = join_path(root, 'Results', 'CalendarData.xlsx')

DataFrame(daily_mean_values).to_excel(excel_filename, index = False)

## 2.2 PM 2.5 Concentration: Map

In [5]:
quarter_mins = {}
quarter_maxs = {}
for type_ in ['wholeDay', 'daytime', 'nighttime']:
    raster_name = join_path(root, 'Plot.gdb', f'PM25_2021_{type_}')
    raster = arcpy.Raster(raster_name)
    quarter_mins['annual'] = min(quarter_mins['annual'], raster.minimum) if 'annual' in quarter_mins else raster.minimum
    quarter_maxs['annual'] = max(quarter_maxs['annual'], raster.maximum) if 'annual' in quarter_maxs else raster.maximum

for quarter in range(1, 5):
    for type_ in ['wholeDay', 'daytime', 'nighttime']:
        raster_name = join_path(root, 'Plot.gdb', f'PM25_2021Q{quarter}_{type_}')
        raster = arcpy.Raster(raster_name)
        quarter_mins[quarter] = min(quarter_mins[quarter], raster.minimum) if quarter in quarter_mins else raster.minimum
        quarter_maxs[quarter] = max(quarter_maxs[quarter], raster.maximum) if quarter in quarter_maxs else raster.maximum

for key in quarter_mins:
    quarter_mins[key] = math.floor(quarter_mins[key])
    quarter_maxs[key] = math.ceil(quarter_maxs[key])

In [6]:
for type_ in ['wholeDay', 'daytime', 'nighttime']:
    map_name = f'PM25_2021_{type_}'
    raster_name = join_path(root, 'Plot.gdb', f'PM25_2021_{type_}')
    border_name = join_path(root, 'Publish.gdb', 'MainUrbanArea')

    if aprx.listMaps(map_name):
        map_visualization = aprx.listMaps(map_name)[0]
    else:
        map_visualization = aprx.createMap(map_name, 'MAP')

    for layer in map_visualization.listLayers():
        map_visualization.removeLayer(layer)

    map_visualization.spatialReference = arcpy.SpatialReference(2381)

    raster = map_visualization.addDataFromPath(raster_name)
    border = map_visualization.addDataFromPath(border_name)

    apply_border_renderer(border, 'Black Outline (1pt)')
    apply_raster_colorizer(
        raster, 
        'Spectrum By Wavelength-Full Bright', 
        quarter_mins['annual'], 
        quarter_maxs['annual']
    )

In [8]:
for quarter in range(1, 5):
    for type_ in ['wholeDay', 'daytime', 'nighttime']:
        map_name = f'PM25_2021Q{quarter}_{type_}'
        raster_name = join_path(root, 'Plot.gdb', f'PM25_2021Q{quarter}_{type_}')
        border_name = join_path(root, 'Publish.gdb', 'MainUrbanArea')
        
        if aprx.listMaps(map_name):
            map_visualization = aprx.listMaps(map_name)[0]
        else:
            map_visualization = aprx.createMap(map_name, 'MAP')
        
        for layer in map_visualization.listLayers():
            map_visualization.removeLayer(layer)
            
        map_visualization.spatialReference = arcpy.SpatialReference(2381)
        
        raster = map_visualization.addDataFromPath(raster_name)
        border = map_visualization.addDataFromPath(border_name)

        apply_border_renderer(border, 'Black Outline (1pt)')
        apply_raster_colorizer(
            raster, 
            'Spectrum By Wavelength-Full Bright', 
            quarter_mins[quarter], 
            quarter_maxs[quarter]
        )

In [ ]:
'''
layout = aprx.listLayouts('PM25_concentration')[0]
layout_cim = layout.getDefinition('V3')

elements = []
for type_ in ['wholeDay', 'daytime', 'nighttime']:
    uri = f'CIMPATH=pm25_2021_{type_}/pm25_2021_{type_}.json'

    map_frame = arcpy.cim.CreateCIMObjectFromClassName('CIMMapFrame', 'V3')
    map_frame.uRI = uri
    map_frame.anchor = 'BottomLeftCorner'
    map_frame.name = f'MapFrame 2021 {type_}'
    map_frame.visible = True
    
    elements.append(map_frame)
    
for quarter in range(1, 5):
    for type_ in ['wholeDay', 'daytime', 'nighttime']:
        uri = f'CIMPATH=pm25_2021q{quarter}_{type_}/pm25_2021q{quarter}_{type_}.json'
        
        map_frame = arcpy.cim.CreateCIMObjectFromClassName('CIMMapFrame', 'V3')
        map_frame.uRI = uri
        map_frame.anchor = 'BottomLeftCorner'
        map_frame.name = f'MapFrame 2021Q{quarter} {type_}'
        map_frame.visible = True
        
        elements.append(map_frame)
    
layout_cim.elements = elements[0]
layout.setDefinition(layout_cim)
'''

## 2.3 Independent Variables

### 2.3.1 The Percentage of the Non-local Residents

In [94]:
for quarter in range(1, 5):
    map_name = f'Distribution_2021Q{quarter}_NonLocal'
    feature_class = JoinPath(root, 'Gwr.gdb', f'Gwr_2021Q{quarter}')

    if aprx.listMaps(map_name):
        map_visualization = aprx.listMaps(map_name)[0]
    else:
        map_visualization = aprx.createMap(map_name, 'MAP')

    for layer in map_visualization.listLayers():
        map_visualization.removeLayer(layer)

    map_visualization.spatialReference = arcpy.SpatialReference(2381)

    border = map_visualization.addDataFromPath(border_name)
    feature = map_visualization.addDataFromPath(feature_class)
    
    apply_feature_renderer(
        feature, 
        'Prediction', 
        'NonLocal', 
        10
    )
    apply_border_renderer(
        border, 
        'Extent Gray Hollow', 
        width = 0.35
    )



### 2.3.2 House Price

In [96]:
map_name = 'Distribution_2021_HousePrice'
feature_class = JoinPath(root, 'Gwr.gdb', f'Gwr_2021Q1')

if aprx.listMaps(map_name):
    map_visualization = aprx.listMaps(map_name)[0]
else:
    map_visualization = aprx.createMap(map_name, 'MAP')

for layer in map_visualization.listLayers():
    map_visualization.removeLayer(layer)

map_visualization.spatialReference = arcpy.SpatialReference(2381)

border = map_visualization.addDataFromPath(border_name)
feature = map_visualization.addDataFromPath(feature_class)

apply_feature_renderer(
    feature, 
    'Prediction', 
    'HousePrice', 
    10
)
apply_border_renderer(
    border, 
    'Extent Gray Hollow', 
    width = 0.35
)

## 2.4 Local Moran's I

In [74]:
for quarter in range(1, 5):
    map_name = f'LocalMoransI_2021Q{quarter}_NonLocal'
    feature_class = JoinPath(root, 'Gwr.gdb', f'LocalMoransI_2021Q{quarter}_NonLocal')
    border_name = JoinPath(root, 'Publish.gdb', 'GridShape')

    if aprx.listMaps(map_name):
        map_visualization = aprx.listMaps(map_name)[0]
    else:
        map_visualization = aprx.createMap(map_name, 'MAP')

    for layer in map_visualization.listLayers():
        map_visualization.removeLayer(layer)

    map_visualization.spatialReference = arcpy.SpatialReference(2381)

    border = map_visualization.addDataFromPath(border_name)
    feature = map_visualization.addDataFromPath(feature_class)
    
    apply_border_renderer(
        border, 
        'Extent Gray Hollow', 
        width = 0.35
    )


In [75]:
map_name = f'LocalMoransI_2021_HousePrice'
feature_class = JoinPath(root, 'Gwr.gdb', 'LocalMoransI_2021_HousePrice')
border_name = JoinPath(root, 'Publish.gdb', 'GridShape')

if aprx.listMaps(map_name):
    map_visualization = aprx.listMaps(map_name)[0]
else:
    map_visualization = aprx.createMap(map_name, 'MAP')

for layer in map_visualization.listLayers():
    map_visualization.removeLayer(layer)

map_visualization.spatialReference = arcpy.SpatialReference(2381)

border = map_visualization.addDataFromPath(border_name)
feature = map_visualization.addDataFromPath(feature_class)

apply_border_renderer(
    border, 
    'Extent Gray Hollow', 
    width = 0.35
)

## 2.5 GWR

In [16]:
for formula in ['NonLocal.Day', 'NonLocal.Night', 'HousePrice']:
    map_name = f'GwrModel_2021_Merged_{formula}'
    gwr_name = join_path(root, 'Gwr.gdb', map_name.replace('.', '_'))

    if aprx.listMaps(map_name):
        map_visualization = aprx.listMaps(map_name)[0]
    else:
        map_visualization = aprx.createMap(map_name, 'MAP')

    for layer in map_visualization.listLayers():
        map_visualization.removeLayer(layer)

    map_visualization.spatialReference = arcpy.SpatialReference(2381)

    gwr = map_visualization.addDataFromPath(gwr_name)

In [18]:
for quarter in range(1, 5):
    for formula in ['NonLocal.Day', 'NonLocal.Night', 'HousePrice']:
        map_name = f'GwrModel_2021Q{quarter}_{formula}'
        gwr_name = join_path(root, 'Gwr.gdb', map_name.replace('.', '_'))

        if aprx.listMaps(map_name):
            map_visualization = aprx.listMaps(map_name)[0]
        else:
            map_visualization = aprx.createMap(map_name, 'MAP')

        for layer in map_visualization.listLayers():
            map_visualization.removeLayer(layer)

        map_visualization.spatialReference = arcpy.SpatialReference(2381)

        gwr = map_visualization.addDataFromPath(gwr_name)

## 2.6 Data Validation

In [123]:
for month in range(1, 13):
    month_string = '%02d' % month
    map_name = f'Validation_2021{month_string}'
    raster_name = join_path(root, 'Plot.gdb', f'Validation_2021{month_string}')
    border_name = join_path(root, 'Publish.gdb', 'MainUrbanArea')

    if aprx.listMaps(map_name):
        map_visualization = aprx.listMaps(map_name)[0]
    else:
        map_visualization = aprx.createMap(map_name, 'MAP')

    for layer in map_visualization.listLayers():
        map_visualization.removeLayer(layer)

    map_visualization.spatialReference = arcpy.SpatialReference(2381)

    raster = map_visualization.addDataFromPath(raster_name)
    border = map_visualization.addDataFromPath(border_name)

    apply_border_renderer(border, 'Black Outline (1pt)')
    apply_raster_colorizer(
        raster, 
        'Spectrum By Wavelength-Full Bright', 
        -0.5, 
        0.5
    )

In [124]:
for quarter in range(1, 5):
    map_name = f'Validation_2021Q{quarter}'
    raster_name = join_path(root, 'Plot.gdb', f'Validation_2021Q{quarter}')
    border_name = join_path(root, 'Publish.gdb', 'MainUrbanArea')
    
    raster_object = arcpy.Raster(raster_name)
    raster_range = max(abs(raster_object.minimum), abs(raster_object.maximum))

    if aprx.listMaps(map_name):
        map_visualization = aprx.listMaps(map_name)[0]
    else:
        map_visualization = aprx.createMap(map_name, 'MAP')

    for layer in map_visualization.listLayers():
        map_visualization.removeLayer(layer)

    map_visualization.spatialReference = arcpy.SpatialReference(2381)

    raster = map_visualization.addDataFromPath(raster_name)
    border = map_visualization.addDataFromPath(border_name)

    apply_border_renderer(border, 'Black Outline (1pt)')
    apply_raster_colorizer(
        raster, 
        'Spectrum By Wavelength-Full Bright', 
        -raster_range, 
        raster_range
    )